In [ ]:
import os

if "INITIALIZED" not in locals():
    os.getcwd()
    os.chdir("../..")
    INITIALIZED = True

# os.chdir("/mnt/data/code/android/collaborative-intelligence")
print(os.getcwd())

In [ ]:
from operator import itemgetter
from typing import Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.subplots
import plotly.graph_objs as go
import seaborn as sns
import tensorflow as tf
from PIL import Image
from ipywidgets import interact
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import ndimage, signal

import src.analysis.dataset as ds
from src.analysis import plot
from src.analysis.experimentrunner import ExperimentRunner
from src.analysis.utils import (
    basename_of,
    tf_disable_eager_execution,
    title_of,
)

tf_disable_eager_execution()

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
np.set_printoptions(precision=4)

BATCH_SIZE = 4
DATASET_SIZE = 4
data_dir = "data"

NUM_FRAMES = 2

In [ ]:
runner = ExperimentRunner(
    model_name="resnet34",
#     layer_name="conv0",
#     layer_name="bn0",
    layer_name="add_3",
#     layer_name="add_7",
#     model_name="densenet121",
#     layer_name="pool3_pool",
#     layer_name="pool4_pool",
    dataset_size=DATASET_SIZE,
    batch_size=BATCH_SIZE,
)

shape = runner.tensor_layout.shape
dtype = runner.tensor_layout.dtype
h, w, c = shape
x_in_per_cl = 224 / w

In [ ]:
def sample_img() -> np.ndarray:
#     return np.array(Image.open(f"{data_dir}/sample/sample300.jpg"))
    return np.array(Image.open(f"{data_dir}/sample/sample.jpg"))
#     return np.array(Image.open(f"{data_dir}/sample/sample_butterfly.jpg"))

img = sample_img()

In [ ]:
def title_of(
    model_name: str, layer_name: str, layer_i: int, layer_n: int
) -> str:
    return f"{model_name}  {layer_name}" # ({layer_i+1}/{layer_n})"

In [ ]:
def rot(theta: float) -> np.ndarray:
    return np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)],
    ])

def scale(x: float, y: float) -> np.ndarray:
    return np.array([
        [x, 0.],
        [0., y],
    ])

def skew(x: float, y: float) -> np.ndarray:
    return np.array([
        [1., x],
        [y, 1.],
    ])

def trans(x: float, y: float) -> np.ndarray:
    return np.array([
        [1., 0., x],
        [0., 1., y],
    ])

def affine33(mat23: np.ndarray) -> np.ndarray:
    mat33 = np.eye(3, dtype=np.float32)
    x = min(3, mat23.shape[1])
    mat33[:2, :x] = mat23[:2, :x]
    return mat33

def affine23(mat33: np.ndarray) -> np.ndarray:
    return mat33[:2, :3].copy()

def affine_mat_from(
    shear_x=0.0,
    shear_y=0.0, 
    scale_x=1.0,
    scale_y=1.0,
    rot_deg=0.0,
    trans_x=0.0,
    trans_y=0.0,
    shear_x_deg=None,
    shear_y_deg=None,
) -> np.ndarray:
    if shear_x_deg is not None:
        shear_x = np.tan(shear_x_deg * np.pi / 180.0)
    if shear_y_deg is not None:
        shear_y = np.tan(shear_y_deg * np.pi / 180.0)
    
    mat = np.eye(3, dtype=np.float32)
    mat[:2, :2] = (
        rot(rot_deg * np.pi / 180.0)
        @ scale(scale_x, scale_y)
        @ skew(shear_x, shear_y)
    )
    mat[:2, 2] = [trans_x, trans_y]
    
    return mat[:2]

In [ ]:
def fix_coordinate_system(mat, src_shape, out_width, out_height, center=True, leeway=None):
    """
    Applies mat to correct coordinate system.
    
    Leeway controls how much % of the image to zoom into,
    after leaving enough room for rotations.
    """
    mat = affine33(mat)
    
    out_radius = np.sqrt((out_width / 2)**2 + (out_height / 2)**2)
    src_radius = min(src_shape[:2]) / 2
    
    s = 1.0
    if leeway is not None:
        s = out_radius / src_radius / (1 - leeway)
        # s = (1 - leeway) * src_radius / out_radius
    
    # s: Source  coordinate system
    # n: Natural coordinate system
    y, x = np.array(src_shape[:2]) / 2
    t_n_from_s = affine33(scale(x=s, y=-s)) @ affine33(trans(-x, -y))
    t_s_from_n = np.linalg.inv(t_n_from_s)
    mat = t_s_from_n @ mat @ t_n_from_s

    # This is for changing the VIEW only
    if center:
        recenter = affine33(trans(-x, -y))
        rescale = affine33(scale(s, s))
        offset = affine33(trans(out_width / 2, out_height / 2))
        mat = offset @ rescale @ recenter @ mat
    
    return affine23(mat)

def transform(img, mat, out_width, out_height, leeway=0.0):
    """Apply given transformation."""
    mat = fix_coordinate_system(
        mat, img.shape, out_width, out_height, center=True, leeway=leeway
    )
    return cv2.warpAffine(
        img, mat, dsize=(out_width, out_height), flags=cv2.INTER_CUBIC
    )

def predict(ref_tensor, mat):
    # mat = mat.copy()
    mat = fix_coordinate_system(mat, (224, 224), 224, 224, center=False)
    mat[:, 2] /= x_in_per_cl
    # NOTE this doesn't work for some interesting reasons... too much work to fix though
    # h, w = ref_tensor.shape[:2]
    # mat = fix_coordinate_system(mat, (h, w), w, h, center=False)

    pred = np.zeros_like(ref_tensor)
    channels = ref_tensor.shape[-1]
    dsize = ref_tensor.shape[:-1]
    flags = cv2.INTER_CUBIC

    for k in range(channels):
        pred[..., k] = cv2.warpAffine(
            ref_tensor[..., k], mat, dsize=dsize, flags=flags
        )

    return pred

def run_experiment(img, transform_params_list, predict, transform, runner, leeway=0.0):
    """Analyzes predictability of tensor data w.r.t. input transforms.

    Determine how well we can predict a target frame's tensor data based
    on only the reference tensor and some transform parameters that
    describe the transform from reference frame to current frame.
    
     - `img` is a reference image to run transform on
     - `transform_params_list` contains args for `transform`
     - `transform` applies `transform_arg` to image frame
     - `predict` applies `tensor_transform` to tensor
       (where `tensor_transform` is computed from `transform_arg`)
    """
    n = len(transform_params_list) + 1
    frames = np.zeros((n, 224, 224, 3), dtype=img.dtype)
    frames[0] = transform(img, affine_mat_from(), 224, 224, leeway=leeway)
    # TODO not very "generic"... affine_mat_from? 224? 
    
    for i, transform_params in enumerate(transform_params_list, start=1):
        frames[i] = transform(img, transform_params, 224, 224, leeway=leeway)

    client_tensors = runner.model_client.predict(frames)
    ref_tensor = client_tensors[0]

    shape = runner.tensor_layout.shape
    dtype = runner.tensor_layout.dtype

    impulse = np.ones(shape, dtype=dtype)
    preds = np.zeros((n - 1, *shape), dtype=dtype)
    masks = np.zeros((n - 1, *shape), dtype=np.bool)

    for i, transform_params in enumerate(transform_params_list):
        preds[i] = predict(ref_tensor, transform_params)
        masks[i] = predict(impulse, transform_params) != 0

    diffs = preds - client_tensors[1:]
    diffs[~masks] = 0
    diffs_masked = diffs
    
    x = client_tensors[1:]
    
    # shape=(n,)
    tensor_ranges = np.max(x, axis=(1, 2, 3)) - np.min(x, axis=(1, 2, 3))
    tensor_counts = np.count_nonzero(masks, axis=(1, 2, 3))
    tensor_sses = np.sum(diffs_masked**2, axis=(1, 2, 3))
    tensor_mses = tensor_sses / tensor_counts
    nrmses = np.sqrt(tensor_mses) / tensor_ranges
    mses = nrmses  # TODO temporarily bypass
    
    # shape=(n, c)
    # chan_ranges = np.max(x, axis=(1, 2)) - np.min(x, axis=(1, 2))
    # chan_counts = np.count_nonzero(masks, axis=(1, 2))
    # chan_sses = np.sum(diffs_masked**2, axis=(1, 2))
    # chan_mses = chan_sses / chan_counts
    
    # shape=(n,)
    # mses = np.sum(chan_sses, axis=-1) / np.sum(chan_counts, axis=-1)
    # psnrs = 10 * np.mean(np.log(chan_ranges ** 2 / chan_mses), axis=-1)
    
    # PSNR doesn't matter anyways
    psnrs = mses
    
    return frames, client_tensors, preds, diffs, mses, psnrs

In [ ]:
img_ = img.copy()
y, x = (np.array(img_.shape[:2]) / 2).astype(int)
z = 3
img_[y-z:y+z, x-z:x+z] = 0

out_width, out_height = 100, 100

mat = affine_mat_from(
    trans_x=50,
    trans_y=0,
    rot_deg=0,
    shear_x_deg=0, shear_y_deg=0,
    scale_x=1, scale_y=1,
)

# frame = transform(img_, mat, out_width, out_height, leeway=-np.sqrt(2) + 1)
frame = transform(img_, mat, out_width, out_height, leeway=0.0)
plt.imshow(frame)

In [ ]:
img_ = img.copy()
y, x = (np.array(img_.shape[:2]) / 2).astype(int)
img_[y, x] = 0

out_width, out_height = 80, 80
# out_width, out_height = 224, 224

# mat = affine_mat_from(
#     trans_x=39, trans_y=0,
#     rot_deg=40,
#     shear_x_deg=40, shear_y_deg=-20,
#     scale_x=0.8, scale_y=1.1,
# )
mat = affine_mat_from(
    trans_x=39, trans_y=0,
    rot_deg=40,
    shear_x_deg=40, shear_y_deg=-20,
    scale_x=0.8, scale_y=1.1,
)
# frame = transform(img_, mat, out_width, out_height, leeway=-np.sqrt(2) + 1)
frame = transform(img_, mat, out_width, out_height, leeway=0.0)
plt.imshow(frame)

In [ ]:
interact_kwargs = dict(
    shear_x=(-0.2, 0.2, 0.02),
    shear_y=(-0.2, 0.2, 0.02),
    scale_x=(0.5, 2.0, 0.1),
    scale_y=(0.5, 2.0, 0.1),
    rot_deg=(-180.0, 180.0, 1.0),
    trans_x=(-300.0, 300.0, 0.1),
    trans_y=(-300.0, 300.0, 0.1),
)

def update_func(
    shear_x=0.0,
    shear_y=0.0,
    scale_x=1.0,
    scale_y=1.0,
    rot_deg=0.0,
    trans_x=0.0,
    trans_y=0.0,
    fig=None,
):
    with fig.batch_update():
        mat = affine_mat_from(
            shear_x=shear_x,
            shear_y=shear_y,
            scale_x=scale_x,
            scale_y=scale_y,
            rot_deg=rot_deg,
            trans_x=trans_x,
            trans_y=trans_y,
        )

        transform_args = [mat]
        
        frames, client_tensors, preds, diffs, mses, psnrs = run_experiment(
            img, transform_args, predict, transform, runner
        )

        # Display results

        # Adjust for visual purposes
        tensors_ = client_tensors
        preds_ = preds
        diffs_ = np.abs(diffs)

        # Show only k channels
        koff = 13
        k = 3 ** 2
        tensors_ = tensors_[..., koff : koff + k]
        preds_ = preds_[..., koff : koff + k]
        diffs_ = diffs_[..., koff : koff + k]

        # Scale colorbar in a consistent manner
        clim = (tensors_.min(), tensors_.max())
        r = clim[1] - clim[0]
        clim_diff = (0, r)

        # Manual overrides
        clim = (-1.6, 1.4)
        clim_diff = (0, 3.0)
        
        # Plot stuff
        fill_value = clim[0] if clim is not None else None
        order = "hwc"

        tensors_ = plot.featuremap_image(tensors_[1], order, fill_value=fill_value)
        preds_ = plot.featuremap_image(preds_[0], order, fill_value=fill_value)
        diffs_ = plot.featuremap_image(diffs_[0], order, fill_value=fill_value)

        image, heatmap, p, d = fig.data
        image.z = frames[1]
        heatmap.z = tensors_[::-1, :]
        p.z = preds_[::-1, :]
        d.z = diffs_[::-1, :]
        
        fig.update_layout(title=f"MSE: {mses[0]:.3f}")
        # fig.update_layout(title=f"MSE: {mses[0]:.3f}    PSNR: {psnrs[0]:.1f} dB")

In [ ]:
frame = np.zeros((224, 224, 3), dtype=np.uint8)
tensor = np.zeros((224, 224), dtype=np.float32)

fig = plotly.subplots.make_subplots(
    rows=1,
    cols=4,
)

fig.add_traces([
    go.Image(z=frame),
    go.Heatmap(z=tensor, colorscale="Viridis"),
    go.Heatmap(z=tensor, colorscale="Viridis"),
    go.Heatmap(z=tensor, colorscale="Viridis"),
], cols=[1, 2, 3, 4], rows=1)

controllable_fig = go.FigureWidget(fig)
controllable_fig

In [ ]:
@interact(**interact_kwargs)
def update(shear_x=0.0, shear_y=0.0, scale_x=1.0, scale_y=1.0, rot_deg=0.0, trans_x=0.0, trans_y=0.0):
    update_func(shear_x, shear_y, scale_x, scale_y, rot_deg, trans_x, trans_y, fig=controllable_fig)

In [ ]:
# Non-interactive figure here

import importlib
importlib.reload(plot)

def plot_overview():
    transform_kwargs = [
        dict(trans_x=35.0),
        dict(rot_deg=10.0),
        dict(scale_x=1.2),
        dict(shear_x_deg=10.0),
    ]

    transform_args = [
        affine_mat_from(**d) for d in transform_kwargs
    ]

    frames, client_tensors, preds, diffs, mses, psnrs = run_experiment(
        img, transform_args, predict, transform, runner, leeway=1-np.sqrt(2)+0.15,
    )

    def visual_adjust_tensor(t):
        off = t - np.mean(t)
        return np.copysign(np.abs(off) ** 0.5, off) + np.mean(t)

    # Adjust for visual purposes
    tensors_ = visual_adjust_tensor(client_tensors)
    preds_ = visual_adjust_tensor(preds)
    diffs_ = np.abs(diffs)

    # Show only k channels
    koff = 0
    k = 3 ** 2
    tensors_ = tensors_[..., koff : koff + k]
    preds_ = preds_[..., koff : koff + k]
    diffs_ = diffs_[..., koff : koff + k]

    # Scale colorbar in a consistent manner
    clim = (tensors_.min(), tensors_.max())
    r = clim[1] - clim[0]
    clim_diff = (0, r)

    # Manual overrides
    clim = (-1.6, 1.4)
    clim_diff = (0, 3.0)

    fig = plot.featuremapsequence(
        frames,
        tensors_,
        preds_,
        diffs_,
        title="",
        clim=clim,
        clim_diff=clim_diff,
        figsize=(8.0, 10),
    )

    def idx(y, x):
        return 4 * y + x

    pad = 6
    ax_opts = dict(
        xy=(0.5, 1),
        xycoords="axes fraction",
        xytext=(0, pad),
        textcoords="offset points",
        size="medium",
        ha="center",
        va="baseline",
    )
    axes = fig.axes
    axes[idx(0, 0)].annotate("Image", **ax_opts)
    axes[idx(0, 1)].annotate("Tensor", **ax_opts)
    axes[idx(1, 2)].annotate("Motion compensated\ntensor", **ax_opts)
    axes[idx(1, 3)].annotate("Difference", **ax_opts)
    axes[idx(0, 0)].set_ylabel("Reference")
    axes[idx(1, 0)].set_ylabel(f"translate (horizontal)")
    axes[idx(2, 0)].set_ylabel(f"rotate")
    axes[idx(3, 0)].set_ylabel(f"stretch (horizontal)")
    axes[idx(4, 0)].set_ylabel(f"shear (horizontal)")

    plot.save(
        fig,
        f"img/experiment/icassp/{runner.basename}-overview.png",
        bbox_inches="tight",
        dpi=300,
        facecolor="w"
    )
    
    return fig, axes

_ = plot_overview()

In [ ]:
trans_range = 56.0
k = int(trans_range // x_in_per_cl)
a = k * x_in_per_cl
xpass_trans = np.linspace(-a, a, 2 * k + 1)

m = 4
# m = 1
samples = 1 * 2**(m + 6) + 1
samples_trans = int(trans_range) * 2**(m + 0) + 1

experiments = [
    {
        "xlabel": "Rotation angle (degrees)",
        "xkwarg": "rot_deg",
        "x": np.linspace(-180, 180, samples),
        # "x": np.linspace(-30, 30, samples),
        "xpass": [0.0],
    },
    {
        "xlabel": "Shear angle (degrees)",
        "xkwarg": "shear_x_deg",
        "x": np.linspace(-20, 20, samples),
        "xpass": [0.0],
    },
    {
        "xlabel": "Shear angle (degrees)",
        "xkwarg": "shear_y_deg",
        "x": np.linspace(-20, 20, samples),
        "xpass": [0.0],
    },
    {
        "xlabel": "Horizontal scale factor",
        "xkwarg": "scale_x",
        "x": np.linspace(0.5, 2.0, samples),
        "xpass": [1.0],
    },
    {
        "xlabel": "Vertical scale factor",
        "xkwarg": "scale_y",
        "x": np.linspace(0.5, 2.0, samples),
        "xpass": [1.0],
    },
    {
        "xlabel": "Translation (px)",
        "xkwarg": "trans_x",
        "x": np.linspace(-trans_range, trans_range, samples_trans),
        "xpass": xpass_trans,
    },
    {
        "xlabel": "Translation (px)",
        "xkwarg": "trans_y",
        "x": np.linspace(-trans_range, trans_range, samples_trans),
        "xpass": xpass_trans,
    },
]

def run_transform_curve_experiment(experiment):
    experiment = dict(experiment)
    
    x = experiment["x"]
    t = experiment.get("t", experiment["x"])
    
    kwargs = [{experiment["xkwarg"]: v} for v in t]
    transform_args = [
        affine_mat_from(**d) for d in kwargs
    ]
    frames, client_tensors, preds, diffs, mses, psnrs = run_experiment(
        img, transform_args, predict, transform, runner
    )
    y = mses
    experiment["y"] = y
    
    # Smoothen curves
    window = 2 * int(0.075 * x.size // 2) + 1
    y_raw = y
    y = signal.savgol_filter(y, window, 3)
    if "xpass" in experiment:
        x0s = np.array(experiment["xpass"])
        mask = np.isclose(x[:, np.newaxis], x0s[np.newaxis], atol=2 / x.size, rtol=0)
        mask = np.any(mask, axis=-1)
        y[mask] = y_raw[mask]
    experiment["y_smooth"] = y
    
    return experiment
    
def run_transform_curve_experiments(experiments):
    prefix = f"img/experiment/icassp/csv/{runner.basename}-transform_curves"
    results = []
    for experiment in experiments:
        experiment = run_transform_curve_experiment(experiment)
        results.append(experiment)
        df = pd.DataFrame({k: experiment[k] for k in ["x", "y", "y_smooth"]})
        df.to_csv(f"{prefix}-{experiment['xkwarg']}.csv", index=False)
    return results

def plot_single_transform_curve(x, y, xlabel, xkwarg):
    prefix = f"img/experiment/icassp/{runner.basename}-transform_curves"
    filename = f"{prefix}-{xkwarg}.png"
    fig, ax = plt.subplots()
    _ = ax.plot(x, y)
    ax.set_ylim(bottom=0)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("NRMSE")
    fig.savefig(filename, dpi=300, facecolor="w")
    return fig

In [ ]:
model_configs = [
    {"model_name": "resnet34", "layer_i": 3, "layer_n": 37, "layer_name": "conv0"},
    {"model_name": "resnet34", "layer_i": 15, "layer_n": 37, "layer_name": "add_3"},
    {"model_name": "resnet34", "layer_i": 21, "layer_n": 37, "layer_name": "add_7"},
    {"model_name": "densenet121", "layer_i": 32, "layer_n": 81, "layer_name": "pool3_pool"},
    {"model_name": "densenet121", "layer_i": 60, "layer_n": 81, "layer_name": "pool4_pool"},
]

friendly = {
    "resnet34": "ResNet-34",
    "densenet121": "DenseNet-121",
}

xkwargs = [
    "scale_x",
    "shear_x_deg",
    "trans_x",
    "rot_deg",
]

plot_configs = [
    {
        "xlabel": "Rotation angle (degrees)",
        "xkwarg": "rot_deg",
        "xrange": [-10, 10],
    },
    {
        "xlabel": "Shear angle (degrees)",
        "xkwarg": "shear_x_deg",
        "xrange": [-10, 10],
    },
    {
        "xlabel": "Shear angle (degrees)",
        "xkwarg": "shear_y_deg",
        "xrange": [-10, 10],
    },
    {
        "xlabel": "Horizontal scale factor",
        "xkwarg": "scale_x",
        "xrange": [0.8, 1.25],
    },
    {
        "xlabel": "Vertical scale factor",
        "xkwarg": "scale_y",
        "xrange": [0.8, 1.25],
    },
    {
        "xlabel": "Translation (px)",
        "xkwarg": "trans_x",
        "xrange": [-30, 30],
    },
    {
        "xlabel": "Translation (px)",
        "xkwarg": "trans_y",
        "xrange": [-30, 30],
    },
]

all_plot = ["trans_x", "rot_deg", "scale_x", "shear_x_deg"]

def plot_transform_curves():
    png_prefix = f"img/experiment/icassp/transform_curves"

    for plot_config in plot_configs:
        xlabel, xkwarg = itemgetter("xlabel", "xkwarg")(plot_config)
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)

        for model_config in model_configs:
            basename = basename_of(**model_config)
            friendly_config = dict(model_config)
            friendly_config["model_name"] = friendly[friendly_config["model_name"]]
            title = title_of(**friendly_config)
            csv_prefix = f"img/experiment/icassp/csv/{basename}-transform_curves"
            
            df = pd.read_csv(f"{csv_prefix}-{xkwarg}.csv")
            if "xrange" in plot_config:
                a, b = plot_config["xrange"]
                df["x"] = df.loc[(a <= df["x"]) & (df["x"] <= b)]
            ax = sns.lineplot(data=df, x="x", y="y_smooth", label=title)
            
        ax.set(xlabel=xlabel, ylabel="NRMSE")
        ax.set_ylim([0.00, 0.04])

        filename = f"{png_prefix}-{xkwarg}.png"
        fig.savefig(filename, dpi=300, facecolor="w")

def plot_transform_curves_all_in_one():
    png_prefix = f"img/experiment/icassp/transform_curves"
    fig, axes = plt.subplots(2, 2, figsize=(12, 8), tight_layout=True)
    
    pcfgs = [next(d for d in plot_configs if d["xkwarg"] == xkwarg) for xkwarg in all_plot]

    for i, plot_config in enumerate(pcfgs):
        xlabel, xkwarg = itemgetter("xlabel", "xkwarg")(plot_config)
        
        ax = axes[int(i // 2), i % 2]
        plt.sca(ax)

        for model_config in model_configs:
            basename = basename_of(**model_config)
            friendly_config = dict(model_config)
            friendly_config["model_name"] = friendly[friendly_config["model_name"]]
            title = title_of(**friendly_config)
            csv_prefix = f"img/experiment/icassp/csv/{basename}-transform_curves"
            
            df = pd.read_csv(f"{csv_prefix}-{xkwarg}.csv")
            if "xrange" in plot_config:
                a, b = plot_config["xrange"]
                df["x"] = df.loc[(a <= df["x"]) & (df["x"] <= b)]
            _ = sns.lineplot(data=df, x="x", y="y_smooth") #, label=title)
            
        ax.set(xlabel=xlabel, ylabel="NRMSE")
        ax.set_ylim([0.00, 0.04])
        ax.tick_params(axis='y', which='minor')
        ax.set_yticks(np.linspace(0.00, 0.04, 5))
        ax.set_yticks(np.linspace(0.005, 0.035, 4), minor=True)
    
    labels = []
    for model_config in model_configs:
        friendly_config = dict(model_config)
        friendly_config["model_name"] = friendly[friendly_config["model_name"]]
        title = title_of(**friendly_config)
        labels.append(title)
    
    # fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0), bbox_transform=fig.transFigure)
    # fig.legend(labels, loc='upper center', bbox_to_anchor=(0.5, 0), bbox_transform=fig.transFigure, ncol=3)
    # fig.legend(labels, loc='upper center', ncol=5)
    # fig.legend(labels, loc='lower center', ncol=5)
    fig.legend(labels, loc='lower center', ncol=5, bbox_to_anchor=(0.5, -0.05),  bbox_transform=fig.transFigure)

    filename = f"{png_prefix}-all.png"
    fig.savefig(filename, dpi=300, bbox_inches='tight', facecolor="w")

In [ ]:
# experiments = run_transform_curve_experiments(experiments)

# for experiment in experiments:
#     getter = itemgetter("x", "y_smooth", "xlabel", "xkwarg")
#     fig = plot_single_transform_curve(*getter(experiment))

In [ ]:
plot_transform_curves_all_in_one()
plot_transform_curves()

In [ ]:
param_ranges = {
    "shear_x_deg": {"min": -5.0, "max": 5.0},
    "shear_y_deg": {"min": -5.0, "max": 5.0},
    "scale_x": {"min": 0.95, "max": 1.05},
    "scale_y": {"min": 0.95, "max": 1.05},
    "rot_deg": {"min": -10.0, "max": 10.0},
    "trans_x": {"min": -32.0, "max": 32.0},
    "trans_y": {"min": -32.0, "max": 32.0},
}

def rand_params(param_ranges):
    return {
        k: np.random.rand() * (r["max"] - r["min"]) + r["min"]
        for k, r in param_ranges.items()
    }

def run_random_mse_experiment(batches=32, batch_size=64):
    results = []
    
    for _ in range(batches):
        params = [rand_params(param_ranges) for _ in range(batch_size)]
        transform_args = [
            affine_mat_from(**p) for p in params
        ]

        frames, client_tensors, preds, diffs, mses, psnrs = run_experiment(
            img, transform_args, predict, transform, runner
        )
        
        xs = [{"mse": mse, **p} for p, mse in zip(params, mses)]
        results.extend(xs)
    
    df = pd.DataFrame(results)
    prefix = f"img/experiment/icassp/csv/{runner.basename}-random_mse"
    df.to_csv(f"{prefix}.csv", index=False)
    
    return df

def plot_simple_random_mse_experiment(df):
    mses = df["mse"].to_numpy()
    
    arr = mses
    clip_range = None
    bins = 20

    arr = arr.reshape((-1,))
    if clip_range is None:
        clip_range = (np.min(arr), np.max(arr))
    bins_ = np.linspace(*clip_range, bins)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.hist(arr, bins=bins_, density=True)
    ax.set_xlabel("NRMSE")
    ax.set_ylabel("Density")
    fig.savefig(f"img/experiment/icassp/{runner.basename}-mse-histogram.png", dpi=300, facecolor="w")

    fig = plt.figure()
    ax = sns.kdeplot(arr)
    ax.set_xlabel("NRMSE")
    fig.savefig(f"img/experiment/icassp/{runner.basename}-mse-density.png", dpi=300, facecolor="w")
    
def plot_random_mse_histograms():
    png_prefix = f"img/experiment/icassp/random_mse"

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    labels = []

    for model_config in model_configs:
        basename = basename_of(**model_config)
        friendly_config = dict(model_config)
        friendly_config["model_name"] = friendly[friendly_config["model_name"]]
        title = title_of(**friendly_config)
        csv_prefix = f"img/experiment/icassp/csv/{basename}-random_mse"

        df = pd.read_csv(f"{csv_prefix}.csv")
        ax = sns.kdeplot(data=df, x="mse", label=title)
        labels.append(title)

    ax.set(xlabel="NRMSE")
    plt.legend(labels)

    filename = f"{png_prefix}.png"
    fig.savefig(filename, dpi=300, facecolor="w")

In [ ]:
# df = run_random_mse_experiment()
# plot_simple_random_mse_experiment(df)

In [ ]:
plot_random_mse_histograms()

In [ ]:
# Perhaps the "error model" is roughly the maximum of the decomposed transformation MSEs
# Sample space: random (uniform) distributions on params

def error_model(reference_curves, **kwargs):
    preds = []
    for k, v in kwargs.items():
        x, y = reference_curves[k]
        idx = np.abs(x - v).argmin()
        preds.append(y[idx])
    preds = np.sort(preds)[::-1]
    return preds[0] + 0.8 * preds[1], preds
    # return max(preds)

def run_error_model_experiment(experiments):
    reference_curves = {e["xkwarg"]: (e["x"], e["y"]) for e in experiments}

    for k in param_ranges:
        x = reference_curves[k][0]
        z = param_ranges[k]
        assert x.min() <= z["min"]
        assert x.max() >= z["max"]

    pred_mses = []
    preds_mses = []
    transform_args = []

    for i in range(16):
        params = {
            k: np.random.rand() * (r["max"] - r["min"]) + r["min"]
            for k, r in param_ranges.items()
        }
        pred_mse, preds_mse = error_model(reference_curves, **params)
        mat = affine_mat_from(**params)
        pred_mses.append(pred_mse)
        preds_mses.append(preds_mse)
        transform_args.append(mat)

    pred_mses = np.array(pred_mses)
    preds_mses = np.array(preds_mses)

    frames, client_tensors, preds, diffs, mses, psnrs = run_experiment(
        img, transform_args, predict, transform, runner
    )

    diffs = (pred_mses - mses)
    pdiffs = (pred_mses - mses) / mses
    print("Predicting MSE of an affine transformation\n")
    print(f"diff  mean: {np.mean(diffs):.3f}")
    print(f"diff  std:  {np.std(diffs):.3f}")
    print("")
    print(f"%diff mean: {np.mean(pdiffs):.3f}")
    print(f"%diff std:  {np.std(pdiffs):.3f}")
    print("")

    from itertools import islice

    print("actual pred   %diff  predictors")
    for actual_mse, pred_mse, preds_mse in islice(zip(mses, pred_mses, preds_mses), 0, 10):
        print(
            f"{actual_mse:.3f}  {pred_mse:.3f} "
            f"{(actual_mse - pred_mse) / actual_mse:>6.3f}  "
            # f"{(actual_mse - pred_mse) / preds_mse[1]:.3f}"
            f"{np.round(preds_mse, 3)[:2]} "
        )

# Looks like when TWO or more predictions are similarly significant, then MSE is roughly close to their sum?
# Or perhaps we should pick the two most significant contributors and take some sort of weighted sum of them?
# But does this generalize to other images?
# Perhaps an error prediction range is good enough... [a, a + b]
# Or we can report the mean error estimate and the error within the error being approximately the mean/2... :)

In [ ]:
# run_error_model_experiment(experiments)

In [ ]:
for layer in runner.model_client.layers:
    # print(layer.get_config(), layer.get_weights())
    weights = layer.get_weights()
    shapes = [x.shape for x in weights]
    shapes_str = "..." if shapes == [] else str(shapes)
    print(layer.name + "\n" + shapes_str)
    for w in weights:
        if len(w.shape) == 4:
            print(w[:, :, 0, 0])
            print(np.mean(w, axis=(-1, -2)))
#             print(np.std(w, axis=(-1, -2)))
            print(np.max(np.abs(w), axis=(-1, -2)))

    print("")

In [ ]:
img_impulse = np.zeros((224, 224, 3), dtype=np.uint8)
# img_impulse[112, 112] = 255
img_impulse[:, 112] = 255
img_impulse[112, :] = 255

frames = img_impulse[np.newaxis, ...]
client_tensors = runner.model_client.predict(frames)
t = client_tensors[0]

t = t[..., 13:13+9]

title = "cross_1px"
fig = plot.featuremap(t, title, clim=(-2, 2))

In [ ]:
plot.save(fig, f"img/experiment/icassp/tensor_{title}.png", bbox_inches='tight', facecolor="w")

In [ ]:
import cv2

def write_video_h264(filename, frames, is_color=True):
    height, width = 224, 224
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(
        filename, fourcc, fps=10, frameSize=(width, height), isColor=is_color
    )

    for frame in frames:
        writer.write(frame)

    writer.release()

def generate_videos(experiments):
    for experiment in experiments:
        xkwarg, x = itemgetter("xkwarg", "x")(experiment)
        for xkwarg_val in x:
            basename = f"{xkwarg}={xkwarg_val:0.4f}"
            filename = f"icassp_baseline/input/{basename}.mp4"
            mat_id = affine_mat_from()
            img_id = transform(img, mat_id, 224, 224)
            mat = affine_mat_from(**{xkwarg: xkwarg_val})
            img_trans = transform(img, mat, 224, 224)
            frames = [img_id, img_trans]
            # print(filename)
            write_video_h264(filename, frames)

In [ ]:
trans_range = 56.0
k = int(trans_range // x_in_per_cl)
a = k * x_in_per_cl
xpass_trans = np.linspace(-a, a, 2 * k + 1)

m = 0
# m = 1
samples = 1 * 2**(m + 6) + 1
samples_trans = int(trans_range) * 2**(m + 0) + 1

experiments_ = [
    {
        "xlabel": "Rotation angle (degrees)",
        "xkwarg": "rot_deg",
        "x": np.linspace(-180, 180, samples),
        # "x": np.linspace(-30, 30, samples),
        "xpass": [0.0],
    },
    {
        "xlabel": "Shear angle (degrees)",
        "xkwarg": "shear_x_deg",
        "x": np.linspace(-20, 20, samples),
        "xpass": [0.0],
    },
    {
        "xlabel": "Shear angle (degrees)",
        "xkwarg": "shear_y_deg",
        "x": np.linspace(-20, 20, samples),
        "xpass": [0.0],
    },
    {
        "xlabel": "Horizontal scale factor",
        "xkwarg": "scale_x",
        "x": np.linspace(0.5, 2.0, samples),
        "xpass": [1.0],
    },
    {
        "xlabel": "Vertical scale factor",
        "xkwarg": "scale_y",
        "x": np.linspace(0.5, 2.0, samples),
        "xpass": [1.0],
    },
    {
        "xlabel": "Translation (px)",
        "xkwarg": "trans_x",
        "x": np.linspace(-trans_range, trans_range, samples_trans),
        "xpass": xpass_trans,
    },
    {
        "xlabel": "Translation (px)",
        "xkwarg": "trans_y",
        "x": np.linspace(-trans_range, trans_range, samples_trans),
        "xpass": xpass_trans,
    },
]

def generate_videos(experiments):
    for experiment in experiments:
        xkwarg, x = itemgetter("xkwarg", "x")(experiment)
        frames = []
        filename = f"img/experiment/icassp/{xkwarg}_tensor.mp4"
        print(filename)
        for xkwarg_val in x:
            mat_id = affine_mat_from()
            img_id = transform(img, mat_id, 224, 224)
            mat = affine_mat_from(**{xkwarg: xkwarg_val})
            img_trans = transform(img, mat, 224, 224)
            frames.append(img_trans)
        frames = np.array(frames)
        client_tensors = runner.model_client.predict(frames)
        t = client_tensors[..., 0][..., np.newaxis]
        t = t.astype(np.uint8)
        print(len(t))
        write_video_h264(filename, t, is_color=False)

generate_videos(experiments_)

In [ ]:
# generate_videos(experiments)

In [ ]:
# Next, extract motion vectors
# Then, compute the frames predicted ONLY from motion vectors
# MASK OUT INVALID REGIONS BEFORE COMPUTING NRMSE!

# TODO

In [ ]:
import numba

@numba.jit(nopython=True)
def estimate_flow_blockmatching(prev_img, next_img, block_size, search_window):
    """Estimate motion vectors using block matching.

    Args:
        prev_img: previous frame
        next_img: next frame
        block_size: size of blocks to match; must be odd integer
        search_window: size of square to search; must be odd integer
    """
    h, w = prev_img.shape[:2]

    assert prev_img.shape == next_img.shape
    assert h >= block_size and w >= block_size
    assert block_size % 2 != 0
    assert search_window % 2 != 0

    block_radius = (block_size - 1) // 2
    search_radius = (search_window - 1) // 2

    br = block_radius
    sr = search_radius

    flow = np.zeros((h, w, 2), dtype=np.float64)
    err = np.zeros((search_window, search_window), dtype=np.float64)

    for y in range(br, h - br):
        for x in range(br, w - br):
            prev_block = prev_img[y - br : y + br + 1, x - br : x + br + 1]
            
            # Calculate errors between blocks within search window.
            # Search window is within intervals [y2_min, y2_max) and [x2_min, x2_max).
            err[:] = np.inf
            y2_min = max(0, y - sr - br) + br
            x2_min = max(0, x - sr - br) + br
            y2_max = min(h, y + sr + br + 1) - br
            x2_max = min(w, x + sr + br + 1) - br
            
            for y2 in range(y2_min, y2_max):
                for x2 in range(x2_min, x2_max):
                    next_block = next_img[y2 - br : y2 + br + 1, x2 - br : x2 + br + 1]
                    ssd = ((next_block - prev_block)**2).sum()
                    err[y2 - y + sr, x2 - x + sr] = float(ssd)

            # Choose block with lowest error.
            i_best = err.argmin()
            y_best = i_best // search_window - sr
            x_best = i_best % search_window - sr
            
            flow[y, x, 0] = x_best
            flow[y, x, 1] = y_best

    return flow

In [ ]:
def plot_quiver(ax, flow, spacing, margin=0, **kwargs):
    """Plot less dense quiver field.

    Args:
        ax: Matplotlib axis
        flow: motion vectors
        spacing: space (px) between each arrow in grid
        margin: width (px) of enclosing region without arrows
    """
    h, w = flow.shape[:2]

    nx = int((w - 2 * margin) / spacing)
    ny = int((h - 2 * margin) / spacing)

    x = np.linspace(margin, w - margin - 1, nx, dtype=np.int64)
    y = np.linspace(margin, h - margin - 1, ny, dtype=np.int64)

    flow = flow[y][:, x]
    u = flow[:, :, 0]
    v = -flow[:, :, 1]

    ax.quiver(x, y, u, v, **kwargs)


# Load images and tensors.
filenames  = [
    "/mnt/data/code/android/collaborative-intelligence/data/sample/car/1_crop_224.jpg",
    "/mnt/data/code/android/collaborative-intelligence/data/sample/car/2_crop_224.jpg",
    # "/mnt/data/code/android/collaborative-intelligence/data/sample/taxi/sample_frame1_224.jpg",
    # "/mnt/data/code/android/collaborative-intelligence/data/sample/taxi/sample_frame2_224.jpg",
    # "/mnt/data/code/android/collaborative-intelligence/data/sample/pinwheel/sample_frame1_224.jpg",
    # "/mnt/data/code/android/collaborative-intelligence/data/sample/pinwheel/sample_frame2_224.jpg",
]

frames = np.array([np.array(Image.open(x)) for x in filenames])
img_n, img_h, img_w = frames.shape[:3]

if len(frames.shape) == 3:
    # If grayscale, convert to RGB.
    frames = np.broadcast_to(frames[..., np.newaxis], (*frames.shape, 3))

grays = (
    np.array([
        cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY),
        cv2.cvtColor(frames[1], cv2.COLOR_RGB2GRAY),
    ])
    if len(frames.shape) == 4
    else frames.copy()
)

client_tensors = runner.model_client.predict(frames)
n, h, w, c = client_tensors.shape

# Pre-process client tensors.
# from scipy import ndimage
# for i in range(n):
#     for j in range(c):
#         client_tensors[i, ..., j] = ndimage.median_filter(client_tensors[i, ..., j], size=3)

# Rescale client tensors.
# client_tensors_resized = np.zeros((n, img_h, img_w, c), dtype=client_tensors.dtype)
# for i in range(n):
#     for j in range(c):
#         client_tensors_resized[i, ..., j] = cv2.resize(
#             client_tensors[i, ..., j], (img_h, img_w), cv2.INTER_CUBIC
#         )
# client_tensors = client_tensors_resized
# h, w = img_h, img_w

# Calculate flow on video frames.
# flow = cv2.calcOpticalFlowFarneback(
#     # Suggested: 0.5, 3, 15, 3, 5, 1.2, 0
#     grays[0],
#     grays[1],
#     flow=None,
#     pyr_scale=0.5,
#     levels=3,
#     winsize=15,
#     iterations=3,
#     poly_n=5,
#     poly_sigma=1.2,
#     flags=0,
# )

# Calculate flow on video frames.
flow = estimate_flow_blockmatching(
    grays[0], grays[1], block_size=31, search_window=11
)

# Rescale input image flow to same width and height as tensor channel.
t_flow_expected = np.zeros((h, w, 2), dtype=np.float32)
t_flow_expected[..., 0] = cv2.resize(flow[..., 0], (h, w), cv2.INTER_AREA) / (img_w / w)
t_flow_expected[..., 1] = cv2.resize(flow[..., 1], (h, w), cv2.INTER_AREA) / (img_h / h)

# Calculate flow on client tensors.
client_flows = np.zeros((n - 1, h, w, c, 2), dtype=np.float32)

for i in range(n - 1):
    for j in range(c):
        # Skip non-displayed channels.
        if j < 13 or j >= 13 + 9:
            continue

        t_prev = client_tensors[i + 0, ..., j]
        t_next = client_tensors[i + 1, ..., j]
        t_flow = estimate_flow_blockmatching(
            t_prev, t_next, block_size=5, search_window=3
        )
        client_flows[i, ..., j, :] = t_flow

# Reduce red in input image.
# r = frames[..., 0]
# r[:] = np.maximum(r, grays)

# Or perhaps desaturate?
saturation = 0.4
frames[:] = frames * saturation + (1 - saturation) * grays[..., np.newaxis]

# Select k channels from tensor for plot.
off, k = 13, 4
t = client_tensors[0, ..., off : off + k]
t_flow = client_flows[0, ..., off : off + k, :]

# Tile tensors (and their flows) into 2D image.
t = plot.featuremap_image(t)
t_flow = np.concatenate([
    plot.featuremap_image(t_flow[..., 0])[..., np.newaxis],
    plot.featuremap_image(t_flow[..., 1])[..., np.newaxis],
], axis=-1)

# Adjust featuremap contrast (modify "p" parameter).
p = 1.5
t_min = t.min()
t_max = t.max()
t_min = -2.0
t_max = 1.5
t_min, t_max = (
    t_min + 0.5 * (1 - p) * (t_max - t_min),
    t_max - 0.5 * (1 - p) * (t_max - t_min),
)

# Plot.
color = "#ff4477"
fig, axes = plt.subplots(nrows=1, ncols=2)

axes[0].axis("off")
axes[0].imshow(frames[0])
plot_quiver(
    axes[0],
    flow,
    spacing=13,
    margin=8,
    color=color,
    scale=0.4,
    scale_units="x",
    width=0.005 * 2,
    # headwidth=3,
    # headlength=5,
    minshaft=2,
    minlength=0.5,
)

axes[1].axis("off")
axes[1].imshow(t, vmin=t_min, vmax=t_max)
plot_quiver(
    axes[1],
    t_flow,
    spacing=2.5,
    margin=0,
    color=color,
    scale=0.5,
    scale_units="x",
    width=0.005,
)

filename = f"img/experiment/icassp/{runner.basename}-optical-flow.png"
fig.savefig(filename, dpi=300, facecolor="w", bbox_inches="tight")

# Plot single featuremap.
# fig, ax = plt.subplots()
# ax.axis("off")
# ax.imshow(t, vmin=t_min, vmax=t_max)
# fig.savefig("taxi_featuremap_1.png", dpi=300, facecolor="w", bbox_inches="tight")


# TODO
# further quant results
# exended cite with arxiv number

In [ ]:
# m = tf.keras.models.load_model("models/densenet121/densenet121-full.h5")
# m.summary()
# with open("img/summary/densenet121.txt", "w") as f:
#     m.summary(print_fn=lambda x: f.write(x + "\n"))

In [ ]:
# Main:
#
# Y - plot DenseNet, ResNet multi-layer
#   - plot thesis-like "overview"
# Y - nice looking plots
#   - tikz
#   - youtube supplement
#   - describe experiment, captions

# Experimental fixes:
#
# - rerun experiments on high-resolution image (not 224x224)
# - are we computing MSE? or nMSE? or what? should we do nMAE, MAPE, etc?
# - how do we compare MSE of different models/layers/channels?
# - rerun experiments on larger dataset

# Other work:
#
# - cv2.warpPerspective
# - plot actual tensor and predicted color maps with same colorbar (for easier visual comparison)
# - use impulse responses to discover interesting info
# - determine a "noise model"? (Applications: e.g. see if low variance or non-zero mean)
# - determine how these transformations influence all N layers afterwards too (can plot on MSE vs param graph)

# Q: How does this generalize to other models?

# Demonstrate:
# - Another (deeper? or shallower) layer of ResNet
# - One more classification model: e.g. DenseNet?

# Deadline: Oct. 21